# Loading the cleaned data

In [31]:
import os
from pathlib import Path
import pandas as pd

In [32]:
DirPpath = Path(os.path.abspath('')).parent # Fetching the current directory path - Specific for ipynb file - For .py: Path(os.path.dirname(os.path.realpath(__file__)).replace("\\", "/"))

PledgesCsvPath = str(DirPpath.absolute()) + "/CleanedData.csv"  

PledgesDf = pd.read_csv(PledgesCsvPath, index_col=0)

# Preprocessing the texts

In [3]:
import string
import re
import contractions

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecaudron001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ecaudron001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecaudron001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ecaudron001\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
def first_clean(text):
    return " ".join(text.split()).replace("_x000D_","")

first_clean(PledgesDf.iloc[10,1]) # Delete the returns from excel

'Working on a monitoring scheme for climate-neutral destinations. Calculating and monitoring the (real! comprehensive) CO2-Footprint of tourism; developing programs to avoid, reduce and compensate. The methodology should be ready, tested and available by end of 2023.'

In [5]:
def ReplaceContractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

ReplaceContractions(PledgesDf.iloc[9,1])

".The Catalan Tourism Data System platform (TDS): is an electronic platform of the online survey system that allows managing and analyzing the evolution of the activity of accommodation and comparing it with the global destination, the type of accommodation and other brands tourist. The target for the next years would be to include expenditure data at the regional level\n•\thttps://empresa.gencat.cat/ca/treb_ambits_actuacio/turisme/coneixement_planificacio/estadistiques-turistiques/tourism-data-system\n•\thttps://catalunya.tourism-data-system.cat/\n\n.The system of indicators of the Tourism Intelligence Indicators System (INTUCAT) has been approved. This index is structured based on 4 areas of measurement, which refer to the environmental sustainability, social sustainability and economic sustainability of the tourism activity at the destination. A fourth area that has been considered essential is the measurement of the achievement of the Sustainable Development Goals (SDGs).\nTarget: 

In [6]:
def RemoveURL(text):
    """Remove URLs from a sample string"""
    text = re.sub(r"https:\s?\S+", "", text)
    return re.sub(r"http\S+", "", text)

RemoveURL(PledgesDf.iloc[5,1])

'Achieve sustainable and flexible solutions for multimodal transport and develop policies to protect natural heritage and biodiversity, respecting the socio-cultural authenticity of host communities.\n\nCNA Turismo e Commercio, over the three-year period (between autumn 2022 and throughout 2023), will organize training seminars for businesses - with the involvement of public and private stakeholders - aimed at the implementation of concrete solutions for the development of good practices for a supply of multimodal transport and protocols for the respect and protection of the natural heritage and biodiversity.'

In [33]:
def preprocess(text):
    text = text.lower() # Lowercase all the characters from the string
    text = text.strip() # Remove the leading and trailing whitespaces
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text) # Removing Punctuation
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text)) # Remove non alphanumeric characters
    text = re.sub(r'\d', '', text) # Removing digits
    text = re.sub(r"\b[a-zA-Z]\b", "", text) # Removing single characters
    text = re.sub(r'\s+', ' ', str(text).strip()) # Replacing "double, triple, etc" whitespaces by one
    return text

preprocess(ReplaceContractions(RemoveURL(first_clean(PledgesDf.iloc[0,1])))) # Testing on pledge 4

'actually we as an association are still pretty much at the beginning due to the pandemic which took the better part of our ressources what we want to provide is proper guideline for str how to achieve maintain and develop sustainable business additionally our business is very fragmented and diverse we have privat hosts with one or only few apartments professional hosts local property managers of all sizes platforms for str local national and international level and service providers for the industry of all kind our target for is to achieve that guideline nonetheless some of our members have implemented variation of measures like strategies for sustainable transmission climate compensation specific advertising for sustainable accomodations using local supplies charging stations for cars using renewable energy and co neutral construction tools to minimise negative impact of tourism on neighbourhoods etc'

In [8]:
def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

stopword(preprocess(ReplaceContractions(RemoveURL(first_clean(PledgesDf.iloc[3,1])))))

'estonia sees need synchronize harmonize rules short term rentals eu level participate process common development implementation common principles however bearing mind regions different may need different approach'

In [9]:
# LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()


# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(nltk.word_tokenize(string))  # Get position tags
    a = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in
         enumerate(word_pos_tags)]  # Map the position tag and lemmatize the word/token
    return " ".join(a)

lemmatizer(stopword(preprocess(first_clean(PledgesDf.iloc[0,1]))))

'actually association still pretty much begin due pandemic take well part ressources want provide proper guideline str achieve maintain develop sustainable business additionally business fragment diverse privat host one apartment professional host local property manager size platform str local national international level service provider industry kind target achieve guideline nonetheless member implement variation measure like strategy sustainable transmission climate compensation specific advertising sustainable accomodations use local supply charge station car use renewable energy co neutral construction tool minimise negative impact tourism neighbourhood etc'

In [ ]:
import gensim
import spacy # python -m spacy download en_core_web_sm = CMD in your terminal

PledgesDf['PreProcessedText'] = PledgesDf['Pledge'].apply(lambda x: preprocess(stopword(ReplaceContractions(RemoveURL(first_clean(x))))))
data_words = [nltk.word_tokenize(i) for i in PledgesDf['PreProcessedText']]


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stopwords.words('english'), allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load("en_core_web_sm")
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return [' '.join(pledge) for pledge in texts_out]

data_ready = process_words(data_words)  # processed Text Data!

In [34]:
def preprocessing(text):

    global n
    n = n +1
    
    print("**************")
    print("n is : ")
    print(n)
    print("length of the text is : ")
    print(len(first_clean(text)))
    return lemmatizer(preprocess(stopword(ReplaceContractions(RemoveURL(first_clean(text))))))

In [ ]:
n=0
print("begin pre-processing")
PledgesDf['clean_text'] = PledgesDf['Pledge'].apply(lambda x: preprocessing(x))
print("end pre-processing")

In [111]:
PledgesDf['clean_text2'] = data_ready

In [13]:
PledgesDf

,Topic,Pledge,Pledge status,clean_text
0,1,Actually we as an association are still pretty...,Published,actually association still pretty much begin d...
1,1,EFFAT welcomes the Commission Proposal for a R...,Ready for publishing,effat welcome commission proposal regulation d...
2,1,HOTREC calls for a level playing field and fai...,Published,hotrec call level play field fair competition ...
3,1,Estonia sees the need to synchronize and harmo...,Published,estonia see need synchronize harmonize rule sh...
4,1,Sphere Travel Club contributes to a flourishin...,Ready for publishing,sphere travel club contribute flourish transpa...
...,...,...,...,...
377,28,Digital Innovation of Cultural Heritage (DICH)...,Published,digital innovation cultural heritage dich over...
378,28,“Lofoten the Green Islands 2030” is a programm...,Published,lofoten green island programme partnership rea...
379,28,To promote health tourism as the unique sellin...,Ready for publishing,to promote health tourism unique selling point...
380,28,As a small tour operator we try to engage our ...,Published,a small tour operator try engage customer make...


# Analysis of pre-processed data

In [ ]:
pd.Series(" ".join(PledgesDf['clean_text']).split()).value_counts()[:30].index.tolist() # Most frequent words

In [ ]:
StopWords2 = pd.Series(" ".join(PledgesDf['clean_text']).split()).value_counts()[:30].index.tolist()

def RemoveFrequentWords(string, FrequentWords):
    a = [i for i in string.split() if i not in FrequentWords] # Removing usual english stopwords from the string
    return ' '.join(a) #Output - Same string after all the transformations

PledgesDf['clean_text'] = PledgesDf['clean_text'].apply(lambda x: RemoveFrequentWords(x, StopWords2))

print(PledgesDf.head())

# Tokenization

In [36]:
tokens = [nltk.word_tokenize(i) for i in PledgesDf["clean_text"]]

In [141]:
word_freq = {}
for sent in tokens:
    for i in sent:

        if i not in word_freq.keys():
            word_freq[i] = 1
        else:
            word_freq[i] += 1
len(word_freq)

3209

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer = TfidfVectorizer(analyzer='word')

tfidf_wm = tfidfvectorizer.fit_transform(PledgesDf["clean_text"])

tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

# Word2Vec

In [16]:
import numpy as np
import gensim.downloader as api

## Pre-trained

In [17]:
wv2 = api.load('word2vec-google-news-300')

### Mean Embedding

In [ ]:
#building Word2Vec model with Average method
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

w2v2 = dict(zip(wv2.index_to_key, wv2.vectors))
modelw2 = MeanEmbeddingVectorizer(w2v2)

# converting text to numerical data using Word2Vec
vectors_w2v2 = modelw2.transform(tokens)

print(vectors_w2v2)

In [39]:
DocIndexV1 = pd.DataFrame(vectors_w2v2)
IndexedPath = str(DirPpath.absolute()) + "\IndexedDataV1.csv"
DocIndexV1.to_csv(IndexedPath)

### Tf-Idf Embedding

In [ ]:
#building Word2Vec model with TF-IDF method
class TfIdfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self

    def transform(self, X, tfidf):

        DocList = []
        i = 0
        
        for words in X:

            WordList = []
            SumWeight = 0 # For applying a sum product appoach

            for w in words:
                 
                try:
                    if w in self.word2vec:
                        weight = tfidf[w].iloc[i]
                        WordList.append(self.word2vec[w] * weight)
                    else:
                        weight = tfidf[w].iloc[i]
                        WordList.append(np.zeros(self.dim))
                except:
                    weight = 0
                    WordList.append(np.zeros(self.dim))

                SumWeight += weight

            i+=1
            DocList.append(np.sum(np.array(WordList), axis = 0) / SumWeight) 
        
        print(np.array(DocList).shape)
        return np.array(DocList)


w2v2 = dict(zip(wv2.index_to_key, wv2.vectors))
modelw2 = TfIdfEmbeddingVectorizer(w2v2)

# converting text to numerical data using Word2Vec
vectors_w2v2 = modelw2.transform(tokens, df_tfidfvect)

print(vectors_w2v2)

In [43]:
DocIndexV1 = pd.DataFrame(vectors_w2v2)
IndexedPath = str(DirPpath.absolute()) + "\IndexedDataV1Tf.csv"
DocIndexV1.to_csv(IndexedPath)


## Fine-Tuning Google 300 news Vectors

In [149]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Setting the model with its hyperparameters
#model = Word2Vec(sentences = tokens, vector_size=300, min_count = 1)
model2 = Word2Vec(sentences = tokens, vector_size=300, min_count = 1)

In [150]:
model = KeyedVectors.load_word2vec_format(r"C:\Users\ecaudron001\Downloads\GoogleNews-vectors-negative300.bin",
                                         binary = True)

In [151]:
import numpy as np

In [152]:
model2.build_vocab(tokens)
total_examples = model2.corpus_count

vocab = list(model2.wv.key_to_index.keys())

In [153]:
model2.build_vocab([list(model.key_to_index.keys())], update=True)


https://datascience.stackexchange.com/questions/97568/fine-tuning-pre-trained-word2vec-model-with-gensim-4-0

In [154]:
model2.wv.vectors_lockf = np.ones(len(model2.wv))

In [155]:
model2.wv.intersect_word2vec_format(r"C:\Users\ecaudron001\Downloads\GoogleNews-vectors-negative300.bin", binary=True)

In [156]:
model2.train(tokens, total_examples=total_examples, epochs=model2.epochs)

(115070, 115070)

In [157]:
model2.wv.most_similar("call")

[('Call', 0.6494125723838806),
 ('calls', 0.6452654004096985),
 ('calling', 0.6199396252632141),
 ('dial_#-###-###', 0.582241415977478),
 ('visit_www.clairmail.com', 0.5597918629646301),
 ('visit_www.oscars.org', 0.5302615761756897),
 ('See_www.thewerehouse.com', 0.5248600244522095),
 ('Charlene_Pellin_answered', 0.5245307683944702),
 ('information_visit_http://www.tempurpedic.com', 0.5230315327644348),
 ('calll', 0.521233081817627)]

In [158]:
model2.save(str(DirPpath.absolute()) + "\Word2VecFt.model")

### Mean Embedding

In [159]:
w2v2 = dict(zip(model2.wv.index_to_key, model2.wv.vectors))
modelw2 = MeanEmbeddingVectorizer(w2v2)

# converting text to numerical data using Word2Vec
vectors_w2v2 = modelw2.transform(tokens)

print(vectors_w2v2)

[[-5.91528341e-02  2.45144889e-02 -9.77118686e-03 ... -2.14853287e-02
  -5.02708368e-03 -1.71505511e-02]
 [-1.31707042e-02 -2.82868720e-03  3.06689576e-03 ... -2.37877597e-03
   2.52336599e-02 -2.39090733e-02]
 [-2.01607645e-02  2.33879872e-02  9.59545281e-03 ... -3.29050049e-02
   3.04455496e-02  1.77179743e-02]
 ...
 [-4.74511497e-02  2.40889918e-02 -2.04083859e-03 ... -3.03844269e-02
   6.61780164e-02 -7.16137365e-05]
 [-1.23493625e-02  1.32553726e-02 -4.54793079e-03 ... -2.89725866e-02
   8.95279124e-02 -1.37805715e-02]
 [-3.62874232e-02  1.73829589e-03 -1.23673230e-02 ... -1.07643884e-02
   2.47869939e-02  1.84420142e-02]]


In [160]:
DocIndexV1 = pd.DataFrame(vectors_w2v2)
IndexedPath = str(DirPpath.absolute()) + "\IndexedDataV2.csv"
DocIndexV1.to_csv(IndexedPath)

### Tf-Idf embedding

In [161]:
w2v2 = dict(zip(model2.wv.index_to_key, model2.wv.vectors))
modelw2 = TfIdfEmbeddingVectorizer(w2v2)

# converting text to numerical data using Word2Vec
vectors_w2v2 = modelw2.transform(tokens, df_tfidfvect)

print(vectors_w2v2)

(252, 300)
[[-0.3396613   1.26389937 -1.31796261 ... -0.53777768 -1.22251308
   1.30233548]
 [-0.17496285  0.07041772 -0.02574716 ... -0.11364882  0.13680986
  -0.14311348]
 [-0.02132123  0.07667647  0.0689479  ... -0.25388294  0.22600214
   0.17822382]
 ...
 [-0.83201844  0.54158151  0.26405451 ... -0.27425581  0.83317399
   0.05091583]
 [ 0.03621177  0.34011728 -0.16661881 ... -0.88733387  3.00252008
   0.23049708]
 [-0.4722026  -0.00477884 -0.30624428 ...  0.10495146  0.26195163
   0.27870798]]


In [162]:
DocIndexV1 = pd.DataFrame(vectors_w2v2)
IndexedPath = str(DirPpath.absolute()) + "\IndexedDataV2Tf.csv"
DocIndexV1.to_csv(IndexedPath)